<a href="https://colab.research.google.com/github/sallymoon10/ResnetImageClassification/blob/master/Kaggle_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import numpy as np
import keras
from keras.layers.core import Dropout, Dense
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import pandas as pd

from keras.optimizers import SGD, Adam

In [0]:
!wget http://dev1-kimia.uwaterloo.ca:5678/data/kaggle/data.zip

--2019-08-18 23:57:39--  http://dev1-kimia.uwaterloo.ca:5678/data/kaggle/data.zip
Resolving dev1-kimia.uwaterloo.ca (dev1-kimia.uwaterloo.ca)... 129.97.229.28
Connecting to dev1-kimia.uwaterloo.ca (dev1-kimia.uwaterloo.ca)|129.97.229.28|:5678... failed: Connection refused.


In [0]:
!unzip -o data.zip

Archive:  data.zip
  inflating: train_x.npy             
  inflating: test_x.npy              
  inflating: sample_submission.csv   
  inflating: train_label.npy         


In [0]:
X_test_npy = np.load('test_x.npy')
X_train_npy = np.load('train_x.npy')
y_train_npy = np.load('train_label.npy')

In [0]:
from keras.utils import np_utils

# Normalize image
classes = np.unique(y_train_npy)
X_test = X_test_npy/255.

X_train = X_train_npy/255.
Y_train = np_utils.to_categorical(y_train_npy, num_classes = len(classes))


plt.imshow(X_train)

In [0]:
from keras import applications

img_height,img_width = 168,308 

base_model = applications.resnet50.ResNet50(include_top=False, input_shape=X_train.shape[1:], weights="imagenet")

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 168, 308, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 174, 314, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 84, 154, 64)  9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 84, 154, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.models import Sequential,Model,load_model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(len(classes), activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.optimizers import SGD, Adam
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
760/760 [==============================] - 40s 53ms/step - loss: 1.9901 - acc: 0.4513
Epoch 2/10
760/760 [==============================] - 24s 31ms/step - loss: 0.3154 - acc: 0.9132
Epoch 3/10
760/760 [==============================] - 24s 32ms/step - loss: 0.1057 - acc: 0.9737
Epoch 4/10
760/760 [==============================] - 24s 32ms/step - loss: 0.0509 - acc: 0.9908
Epoch 5/10
760/760 [==============================] - 24s 32ms/step - loss: 0.0323 - acc: 0.9947
Epoch 6/10
760/760 [==============================] - 24s 32ms/step - loss: 0.0332 - acc: 0.9934
Epoch 7/10
760/760 [==============================] - 24s 32ms/step - loss: 0.0374 - acc: 0.9947
Epoch 8/10
760/760 [==============================] - 24s 32ms/step - loss: 0.0220 - acc: 0.9947
Epoch 9/10
760/760 [==============================] - 24s 32ms/step - loss: 0.0200 - acc: 0.9974
Epoch 10/10
760/760 [==============================] - 24s 32ms/step - loss: 0.

In [0]:
Y_test = model.predict(X_test)

In [0]:
# Save submission file as CSV file
from google.colab import files

with open('submission.csv', 'w') as file:
  file.write('Id,Predicted\n')
  for index, sample in enumerate(Y_test):
    file.write(str(index) + ',' + str(np.argmax(sample)) + '\n')

files.download('submission.csv')